## Transformations used to convert original dataset to match SOPHY

In [5]:
import pandas as pd
import numpy as np
import sqlite3
import geolabel
import sophysql
import sophytaxa
import cartopy.crs as ccrs
import geopandas as gpd
from geopandas import GeoDataFrame
from pandas import DataFrame

In [6]:
con = sqlite3.connect("test.db")
with open('schema.sql', 'r') as sql_file:
    con.executescript(sql_file.read())
con.commit()

1) Palmer LTER dataset
- Link to source and info

In [7]:
lter1 = pd.read_csv("../data/in/datasets/unmodified/AntarcticaLTERcompiledData_Cruise_forEDI.csv")
lter2 = pd.read_csv("../data/in/datasets/unmodified/AntarcticaLTERcompiledData_Station_forEDI.csv")

lter_sql: dict = {"DatetimeGMT": "timestamp", "Latitude": "latitude", "Longitude": "longitude",
                  "Depth": "depth", "Temperature": "temperature", "Salinity": "salinity", "Density": "density",
                  "Chlorophyll": "chl_a", "Fluorescence": "fluorescence", "Phaeopigment": "phaeopigments",
                  "PrimaryProduction": "primary_prod", "studyName": "cruise", "PAR": "par",
                  "Prasinophytes": "prasinophytes", "Cryptophytes": "cryptophytes",
                  "MixedFlagellates": "mixed_flagellates", "Diatoms": "diatoms", "Haptophytes": "haptophytes",
                  "NO3": "nitrate", "NO2": "nitrite", "DIC1": "diss_inorg_carbon", "DOC": "diss_org_carbon",
                  "POC": "part_org_carbon", "SiO4": "silicate", "N": "tot_nitrogen",
                  "PO4": "phosphate", "Notes1": "notes"}

lter1 = lter1[lter_sql.keys()].rename(columns=lter_sql)
lter2 = lter2[lter_sql.keys()].rename(columns=lter_sql)
lter = pd.concat([lter1, lter2])
lter = lter.dropna(subset=['timestamp', 'longitude', 'latitude'])
# TODO: warning if any values were dropped
lter = lter[lter['longitude'].between(-180, 180)]
lter = lter[lter['latitude'] <= -30]
# Group chemtax into three main categories
lter['percent_phaeo'] = lter['haptophytes']
lter['percent_diatom'] = lter['diatoms']
lter['percent_prasinophytes'] = lter['prasinophytes']
lter['percent_other'] = lter['mixed_flagellates'] + lter['cryptophytes']

data_gdf = GeoDataFrame(lter, geometry=gpd.points_from_xy(lter['longitude'], lter['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(geolabel.zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labelled data
lter = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right']))
sectors_series: pd.Series = pd.cut(lter['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
lter = lter.assign(sector=sectors_series)

extra = lter.columns.difference(sophysql.get_table_cols("sample"))
lter["extra_json"] = lter[extra].agg(lambda r: r[r.notna()].to_json(), axis=1)
lter = lter.drop(extra, axis=1)
lter

,timestamp,latitude,longitude,depth,temperature,salinity,chl_a,cruise,par,nitrate,nitrite,silicate,phosphate,percent_phaeo,percent_diatom,percent_other,front_zone,sector,extra_json
0,1991-11-07 00:36:00,-64.83333,-64.05167,2.14998,-1.7618,33.791,0.397000,PD91-09,NaN,NaN,NaN,NaN,NaN,0.000000,0.000603,0.859458,SIZ,BA,"{""cryptophytes"":0.0,""diatoms"":0.000602825,""hap..."
1,1991-11-07 00:36:00,-64.83333,-64.05167,10.48500,-1.7445,33.798,0.298000,PD91-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""notes"":""**HPLC in datazoo file listed as ug\..."
2,1991-11-07 00:36:00,-64.83333,-64.05167,20.44900,-1.6974,33.820,0.265000,PD91-09,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.883952,SIZ,BA,"{""cryptophytes"":0.0,""diatoms"":0.0,""haptophytes..."
3,1991-11-07 00:36:00,-64.83333,-64.05167,30.60000,-1.6188,33.846,NaN,PD91-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""part_org_carbon"":123.16,""tot_nitrogen"":17.0}"
4,1991-11-07 00:36:00,-64.83333,-64.05167,40.54660,-1.4717,33.879,0.284000,PD91-09,NaN,NaN,NaN,NaN,NaN,0.073917,0.388492,0.535099,SIZ,BA,"{""cryptophytes"":0.001472699,""diatoms"":0.388492..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19341,2021-05-04 00:00:00,-64.81500,-64.04050,20.00000,NaN,NaN,0.533521,PAL2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""phaeopigments"":0.113480187}"
19342,2021-05-04 00:00:00,-64.81500,-64.04050,35.00000,NaN,NaN,0.488506,PAL2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""phaeopigments"":0.1093088}"
19343,2021-05-04 00:00:00,-64.81500,-64.04050,50.00000,NaN,NaN,0.430152,PAL2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""phaeopigments"":0.190364512}"
19344,2021-05-04 00:00:00,-64.81500,-64.04050,65.00000,NaN,NaN,0.355125,PAL2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""phaeopigments"":0.081884662}"


In [8]:
lter.to_sql(name='sample', con=con, if_exists='append', index=False)

49225

2) Joy-Warren 2019 dataset
- Link to source and info

In [9]:
"""Writes Joy-Warren 2019 dataset to database"""
joyw: DataFrame = pd.read_csv('../data/in/datasets/modified/joy_warren.csv', encoding='utf-8')
# group by depth and station: station1([1.7, 1.8, 2.1][9.7. 9.8, 10.2]...), station2...
joyw = joyw.groupby([joyw["depth"].pct_change().abs().gt(0.15).cumsum(), "station"]).mean(numeric_only=True)
joyw = joyw.reset_index(level=0, drop=True).reset_index(level=0).sort_values(by='depth')
joyw.sort_values(by=['station', 'depth'])
# --------------------------------------------------
jwchemtax = pd.read_csv('../data/in/datasets/modified/joy_warren_chemtax.csv')
jwchemtax = jwchemtax.dropna().sort_values(by='depth')
# --------------------------------------------------
# Join main sample data with chemtax by matching station and depth
joyw: DataFrame = pd.merge_asof(jwchemtax, joyw, by='station', on='depth', direction='nearest',
                                  tolerance=2).sort_values(by='station')

joyw['timestamp'] = pd.to_datetime(joyw['date'], format='%Y%m%d', errors='coerce').dropna().drop(
    columns=['date', 'time'])
joyw['source_name'] = 'joyw'
joyw['percent_phaeo'] = joyw['haptophytes']
joyw['percent_diatom'] = joyw['diatoms']
joyw['percent_other'] = joyw['chlorophytes'] + joyw['mixed_flagellates'] + joyw['cryptophytes']
# ----------------------------------------------------
# Set id field for sample data so foreign keys for microscopy can be matched
joyw = joyw.reset_index(drop=True)
max_id: int = pd.read_sql("select max(id) from sample", con=con)['max(id)'][0] + 1
joyw['id'] = np.arange(max_id, max_id + len(joyw))
jwmkey = pd.concat([joyw['id'], joyw['station'], joyw['depth']], axis=1).sort_values(by='depth')
# Group extra columns into JSON
extra = joyw.columns.difference(sophysql.get_table_cols("sample"))
joyw["extra_json"] = joyw[extra].agg(lambda r: r[r.notna()].to_json(), axis=1)
joyw = joyw.drop(columns=extra, axis=1)

microscopy = pd.read_csv('../data/in/datasets/modified/joy_warren_microscopy.csv', encoding='utf-8').dropna()
replace: tuple = ('centric', 'pennate', 'unknown diatom', 'dinoflagellate', 'ciliate', 'silicoflagellate')
# are_taxa = rows that have species name we can get taxonomy for (ex: Phaeocystis)
are_taxa = ~microscopy['taxa'].isin(replace)
taxa: DataFrame = pd.read_csv("../data/in/worms/joy_warren_worms.csv", encoding='utf-8').rename(sophytaxa.worms_sql)
taxa.index = microscopy[are_taxa].index
# ----------------------------------
microscopy['aphia_id'] = taxa['AphiaID']
microscopy = microscopy.sort_values(by='depth')
# Join microscopy data with matching id in sample table (by depth and station)
microscopy = pd.merge_asof(microscopy, jwmkey, by='station', on='depth', direction='nearest', tolerance=1)
microscopy = microscopy.rename({'id': 'sample_id', 'taxa': 'name', 'group': 'groups'},
                               axis="columns")
# Remove extra columns
microscopy = microscopy[microscopy.columns.intersection(sophysql.get_table_cols("microscopy"))]

data_gdf = GeoDataFrame(joyw, geometry=gpd.points_from_xy(joyw['longitude'], joyw['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(geolabel.zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labels for zones and sectors
joyw = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right'])).drop_duplicates(subset=['id'])
sectors_series: pd.Series = pd.cut(joyw['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
joyw = joyw.assign(sector=sectors_series)
joyw

,depth,latitude,longitude,temperature,salinity,nitrate,nitrite,phosphate,silicate,timestamp,source_name,percent_phaeo,percent_diatom,percent_other,id,extra_json,front_zone,sector
0,9.8,-62.7033,-69.6903,-0.677414,33.799114,27.78,0.189,1.890,22.600,2014-10-31,joyw,0.478822,0.487265,0.033913,98953,"{""CTDFLUOR_UP"":0.9224714286,""CTDOXY2_UP"":8.049...",ASZ,BA
1,3.5,-62.7033,-69.6903,-0.677300,33.798900,NaN,NaN,NaN,NaN,2014-10-31,joyw,0.423520,0.470371,0.106109,98954,"{""CTDFLUOR_UP"":0.90615,""CTDOXY2_UP"":8.04509,""C...",ASZ,BA
2,76.3,-62.7033,-69.6903,-0.681500,33.799200,NaN,NaN,NaN,NaN,2014-10-31,joyw,0.397119,0.430182,0.172699,98955,"{""CTDFLUOR_UP"":0.8833,""CTDOXY2_UP"":8.05498,""CT...",ASZ,BA
3,24.7,-62.7033,-69.6903,-0.677767,33.799067,NaN,NaN,NaN,NaN,2014-10-31,joyw,0.350910,0.617978,0.031113,98956,"{""CTDFLUOR_UP"":0.9033333333,""CTDOXY2_UP"":8.044...",ASZ,BA
4,100.1,-62.7033,-69.6903,-0.681550,33.800650,NaN,NaN,NaN,NaN,2014-10-31,joyw,0.357956,0.558053,0.083991,98957,"{""CTDFLUOR_UP"":0.86,""CTDOXY2_UP"":8.03891,""CTDO...",ASZ,BA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,9.9,-65.8500,-71.3753,-1.709362,33.805200,28.03,0.198,1.878,NaN,2014-11-21,joyw,0.459095,0.432405,0.108500,99199,"{""CTDFLUOR_UP"":1.5700625,""CTDOXY2_UP"":8.010118...",SIZ,BA
247,99.2,-65.8500,-71.3753,1.409000,34.363800,35.21,0.146,2.383,65.970,2014-11-21,joyw,0.474952,0.525048,0.000000,99200,"{""CTDFLUOR_UP"":0.0202,""CTDOXY2_UP"":4.76686,""CT...",SIZ,BA
248,1.3,-65.8500,-71.3753,-1.687700,33.802200,27.66,0.195,1.853,44.440,2014-11-21,joyw,0.469119,0.411942,0.118939,99201,"{""CTDFLUOR_UP"":1.3039,""CTDOXY2_UP"":8.02862,""CT...",SIZ,BA
249,48.8,-65.8500,-71.3753,-1.443133,33.945317,30.40,0.274,2.058,51.825,2014-11-21,joyw,0.512621,0.421544,0.065835,99202,"{""CTDFLUOR_UP"":0.5041833333,""CTDOXY2_UP"":7.346...",SIZ,BA


In [10]:
joyw.to_sql(name='sample', con=con, if_exists='append', index=False)
microscopy.to_sql(name='microscopy', con=con, if_exists='append', index=False)

137

3) Phytobase
- Link to source and info

In [11]:
phybase: DataFrame = pd.read_csv('../data/in/datasets/mod/phytobase.csv')
phybase['timestamp'] = pd.to_datetime(phybase['timestamp'], errors='coerce')
# get full taxonomy of microscopy data as dataframe
taxa: DataFrame = pd.read_csv('../data/in/worms/phytobase_worms.csv')[['original', 'AphiaID']]
# join on sample and taxonomy (by aphia_id), only keep cols in the occurrence table (filter out order, genus, etc)
phybase = pd.merge(phybase, taxa, left_on='scientificname', right_on='original')
phybase = phybase.rename(columns=sophytaxa.worms_sql).filter(sophysql.get_table_cols("occurrence"))

data_gdf = GeoDataFrame(phybase, geometry=gpd.points_from_xy(phybase['longitude'], phybase['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(geolabel.zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labelled data
phybase = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right']))
sectors_series: pd.Series = pd.cut(phybase['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
phybase = phybase.assign(sector=sectors_series)
phybase

,source_name,aphia_id,latitude,longitude,timestamp,depth,front_zone,sector
0,phytobase,620590.0,-41.0001,-74.4499,2002-04-12,0.0,SAZ,BA
1,phytobase,620590.0,-41.0001,-74.4499,2002-04-12,1.0,SAZ,BA
2,phytobase,620590.0,-41.0001,-74.4499,2002-04-12,10.0,SAZ,BA
3,phytobase,620590.0,-41.0001,-74.4499,2002-04-12,11.0,SAZ,BA
4,phytobase,620590.0,-41.0001,-74.4499,2002-04-12,12.0,SAZ,BA
...,...,...,...,...,...,...,...,...
38278,phytobase,NaN,-39.9900,95.0100,1995-09-29,51.0,STZ,WPO
38279,phytobase,NaN,-43.0000,95.0100,1995-09-28,0.0,STZ,WPO
38280,phytobase,NaN,-43.0000,95.0100,1995-09-28,48.0,STZ,WPO
38281,phytobase,NaN,-31.7500,95.0000,1995-10-02,0.0,STZ,WPO


In [12]:
phybase.to_sql(name='occurrence', con=con, if_exists='append', index=False)

38438

4) Alderkamp dataset
- Link to source and info

In [13]:
pd.read_sql("select * from sample;", con=con)

,id,source_name,cruise,latitude,longitude,timestamp,front_zone,sector,percent_phaeo,percent_diatom,...,chl_a,salinity,temperature,mld,par,nitrate,nitrite,phosphate,silicate,extra_json
0,49477,joyw,None,-62.7033,-69.6903,2014-10-31 00:00:00,ASZ,None,0.478822,0.487265,...,NaN,33.799114,-0.677414,None,NaN,27.78,0.189,1.890,22.600,"{""CTDFLUOR_UP"":0.9224714286,""CTDOXY2_UP"":8.049..."
1,49478,joyw,None,-62.7033,-69.6903,2014-10-31 00:00:00,ASZ,None,0.423520,0.470371,...,NaN,33.798900,-0.677300,None,NaN,NaN,NaN,NaN,NaN,"{""CTDFLUOR_UP"":0.90615,""CTDOXY2_UP"":8.04509,""C..."
2,49479,joyw,None,-62.7033,-69.6903,2014-10-31 00:00:00,ASZ,None,0.397119,0.430182,...,NaN,33.799200,-0.681500,None,NaN,NaN,NaN,NaN,NaN,"{""CTDFLUOR_UP"":0.8833,""CTDOXY2_UP"":8.05498,""CT..."
3,49480,joyw,None,-62.7033,-69.6903,2014-10-31 00:00:00,ASZ,None,0.350910,0.617978,...,NaN,33.799067,-0.677767,None,NaN,NaN,NaN,NaN,NaN,"{""CTDFLUOR_UP"":0.9033333333,""CTDOXY2_UP"":8.044..."
4,49481,joyw,None,-62.7033,-69.6903,2014-10-31 00:00:00,ASZ,None,0.357956,0.558053,...,NaN,33.800650,-0.681550,None,NaN,NaN,NaN,NaN,NaN,"{""CTDFLUOR_UP"":0.86,""CTDOXY2_UP"":8.03891,""CTDO..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49722,99199,joyw,None,-65.8500,-71.3753,2014-11-21 00:00:00,SIZ,BA,0.459095,0.432405,...,NaN,33.805200,-1.709362,None,NaN,28.03,0.198,1.878,NaN,"{""CTDFLUOR_UP"":1.5700625,""CTDOXY2_UP"":8.010118..."
49723,99200,joyw,None,-65.8500,-71.3753,2014-11-21 00:00:00,SIZ,BA,0.474952,0.525048,...,NaN,34.363800,1.409000,None,NaN,35.21,0.146,2.383,65.970,"{""CTDFLUOR_UP"":0.0202,""CTDOXY2_UP"":4.76686,""CT..."
49724,99201,joyw,None,-65.8500,-71.3753,2014-11-21 00:00:00,SIZ,BA,0.469119,0.411942,...,NaN,33.802200,-1.687700,None,NaN,27.66,0.195,1.853,44.440,"{""CTDFLUOR_UP"":1.3039,""CTDOXY2_UP"":8.02862,""CT..."
49725,99202,joyw,None,-65.8500,-71.3753,2014-11-21 00:00:00,SIZ,BA,0.512621,0.421544,...,NaN,33.945317,-1.443133,None,NaN,30.40,0.274,2.058,51.825,"{""CTDFLUOR_UP"":0.5041833333,""CTDOXY2_UP"":7.346..."


5) Garibotti dataset
- Link to source and info